In [1]:
import sys
# print(sys.executable)

In [2]:
import eurostat
from neo4j import GraphDatabase, basic_auth
import neo4j
import pandas as pd
import numpy as np
import time
import os
from dotenv import load_dotenv
from pathlib import Path
from functions import preprocessing_eurostat_data,import_sm_obs
from PyPDF2 import PdfReader
import re

In [3]:
#load the environment variables
dotenv_path = Path('~/.env')
load_dotenv(dotenv_path=dotenv_path)  # This line brings all environment variables from .env into os.environ

# Get variables
SUSTAINGRAPH_URI = os.getenv('SUSTAINGRAPH_URI')
SUSTAINGRAPH_USER = os.getenv('SUSTAINGRAPH_USER')
SUSTAINGRAPH_PASSWORD = os.getenv('SUSTAINGRAPH_PASSWORD')
database_name = os.getenv('DATABASE_NAME')

# Connect to database
driver = GraphDatabase.driver(SUSTAINGRAPH_URI, auth=(SUSTAINGRAPH_USER, SUSTAINGRAPH_PASSWORD))

# Verify connectivity
with driver.session(database=database_name) as session:
    print(session.run("RETURN 'Connected to ' + $db", db=database_name).single()[0])

Connected to neo4j


In [4]:
# Add the year
year='2025'

df_eu = pd.read_excel('Data/3.EU_SDGs/3.EU_Indicator_MPI_PolicyTarget.xlsx', sheet_name='eu_sdg')
df_pt = pd.read_excel('Data/3.EU_SDGs/3.EU_Indicator_MPI_PolicyTarget.xlsx',sheet_name ='policy_target')
df_eu_un = pd.read_excel('Data/3.EU_SDGs/3.EU_Indicator_MPI_PolicyTarget.xlsx',sheet_name ='eu_un')
df_mpi = pd.read_excel('Data/3.EU_SDGs/3.EU_Indicator_MPI_PolicyTarget.xlsx',sheet_name ='mpi')


# Get all the codes in the Eurostat database
toc_df = eurostat.get_toc_df()
toc_df = toc_df.map(lambda x: str(x).lower())

In [5]:
def write_eu_indicators(tx,statement, params_dict):
    tx.run(statement, parameters=params_dict)

def write_batch(tx,statement, params_list):
    tx.run(statement, parameters={"parameters": params_list})

[Eurostat](https://ec.europa.eu/eurostat/web/sdi) is called to regularly monitor progress towards the SDGs in an EU context. For this purpose it coordinated the development of the EU SDG indicator set and keeps it up to date. It also produces regular monitoring reports on progress towards the SDGs in an EU context.

> Create EU_SDG Source node

Before importing the Eurostat data, the Source node with property name: "EU_SDG" is created and all the European SDG indicators will be connected with this node through the relationship 'COMES_FROM'.

In [6]:
records, summary, keys = driver.execute_query("""\
        MERGE (s:Source{name:'EU_SDG'})
        RETURN count(s) as Source_Data
        """,
        database_=database_name)
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

Created 1 nodes in 17 ms.


### Import EU_SDG indicators, series, policy targets, association with UN_SDG indicators

At first, we create the EU_SDG Indicators with their respective series, policy targets and the association with the UN indicators. Then, we create the indicators coming from the European Union regarding the SDGs. There are EU_SDG indicators which are part of EU_SDG indicators.
For this reason, we create the relationship (Indicator)-[:ASSOCIATED_WITH {relevance:'part of',dateOfReport:date($report_date)}]->(Indicator).Regarding the Series nodes, each EU_SDG Indicator has the one Series node with relavant properties with the Indicator node, in aligment with the schema of the SustainGraph.

Details regarding EU SDG Indicators, Policy Targets, Multi-Purpose Indicators (MPIs) and the association between EU and UN goals can be found in the '3.EU_Indicator_MPI_PolicyTarget' excel file. 

Explanatory comments regarding the file:
- *summary sheet*: explanatory comments
- *policy target sheet*:
    -  EU_ind: EU SDG Indicator code
    -  Value: absolute value to which the policy target refers 
    -  Unit: unit of measurement to which the value refers (percent, million)
    -  Operator: comparative/arithmetic operator to which the policy target refers (greater than or equal to, less than or equal to, reduce by)
    -  Year: goal year to which the policy target refers
    -  Reference: publication to which the policy target refers 
    -  Comments: relevant comments as published in the Eurostat database
- *mpi sheet*:
    -  eu_sdg: EU SDG Indicator code
    -  goal_code: Goal to which the EU SDG Indicator code is a multi-purpose indicator
    -  DateOfReport: date of the latest report
    -  Comments: relevant comments (detailed explanation found in *summary* sheet)
- *eu_un sheet*:
    -  eu_sdg: EU SDG Indicator code
    -  un_sdg: corresponding UN SDG Indicator code 
    -  association: type of association between the two indicator codes (identical to, similar to, part of, not available)
- *eu_sdg sheet*:
    -  Goal: EU SDG Goal
    -  Indicator_Code: EU SDG Indicator code corresponding to each Goal
    -  Part_of: whether the EU SDG Indicator is part of another EU SDG Indicator (relevance:'part of' in the ASSOCIATED_WITH) relationship
    -  DateOfReport: date of the latest report
    -  Comments: relevant comments (detailed explanation found in *summary* sheet)


The final graph schema is provided below:
![Alt text](wiki/SustainGraph-Indicators.png)

In [6]:
# Merge the two dataframes 
df = pd.merge(df_eu,toc_df, how='left', left_on='Indicator_Code', right_on='code')
print("Number of EU SDG Indicators to be imported:", len(df))

Number of EU SDG Indicators to be imported: 109


In [8]:
parent_statement= """
    MATCH(pi:Indicator{code:$parent_ind}), (i:Indicator{code:$ind})
    MERGE (i)-[:ASSOCIATED_WITH{relevance:'part of',dateOfReport:date($report_date)}]->(pi)
"""
statement_eu_ind = """
    MATCH (g:Goal{code:$goal}),(so:Source{name:'EU_SDG'})
    MERGE (i:Indicator{code:$ind,description:$desc})
    MERGE (s:Series{code:$ind,description:$desc,
                    dataProviderURL:"https://ec.europa.eu/eurostat/web/sdi/database"})
    MERGE (g)-[:HAS_INDICATOR]->(i)
    MERGE (i)-[:HAS_SERIES]->(s)
    MERGE (i)-[:COMES_FROM]->(so)    
    RETURN *
"""

i = 0
with driver.session(database=database_name) as session:
    for index,row in df.iterrows(): 
        session.execute_write(write_eu_indicators, 
                            params_dict = {
                                'goal': str(row.Goal), 
                                'desc': str(row.title),
                                'ind': str(row.Indicator_Code),
                                'parent_ind': row.Part_of
                            },
                            statement = statement_eu_ind)  
        if (pd.notna(row.Part_of)):
            i +=1
            session.execute_write(write_eu_indicators, 
                            params_dict = { 
                                'ind': str(row.Indicator_Code),
                                'parent_ind': row.Part_of,
                                'report_date':str(row['DateOfReport'])
                                },
                            statement = parent_statement)
expected = df_eu['Part_of'].notna().sum()
print('{} EU Indicators imported as part of the EU Indicators,'.format(i), 'expected:', expected) 

7 EU Indicators imported as part of the EU Indicators, expected: 7


### Policy Targets
The Policy Targets are collected from the annual "EU SDG Indicator set" report  by Eurostat. Policy targets from earlier years are maintained in the SustainGraph, unless there are confilcts or refer to outdated indicators

### MPI Indicators
According to the EU Indicator Set, provided by the European Union each year, a set of indicators 33 of the 101 indicators are multipurpose, meaning they are used to monitor more than one SDG.

### Association with UN
The EU SDG indicator set is aligned where appropriate with the UN list of global indicators. 

#### Constraints

> Property types

In [9]:
def create_constraint(tx,statement):
    tx.run(statement)

In [10]:
statement_pt_value = """ 
CREATE CONSTRAINT pt_value IF NOT EXISTS
FOR (pt:EUPolicyTarget) REQUIRE pt.value :: FLOAT
"""

statement_pt_unit = """ 
CREATE CONSTRAINT pt_unit IF NOT EXISTS
FOR (pt:EUPolicyTarget) REQUIRE pt.unit :: STRING
"""

statement_pt_operator = """ 
CREATE CONSTRAINT pt_operator IF NOT EXISTS
FOR (pt:EUPolicyTarget) REQUIRE pt.operator :: STRING
"""

statement_pt_year = """ 
CREATE CONSTRAINT pt_year IF NOT EXISTS
FOR (pt:EUPolicyTarget) REQUIRE pt.byYear :: INTEGER
"""

statement_pt_ref = """ 
CREATE CONSTRAINT pt_ref IF NOT EXISTS
FOR (pt:EUPolicyTarget) REQUIRE pt.reference :: STRING
"""

statement_pt_comment = """ 
CREATE CONSTRAINT pt_comment IF NOT EXISTS
FOR (pt:EUPolicyTarget) REQUIRE pt.comment :: STRING
"""

statement_pt_date = """ 
CREATE CONSTRAINT pt_date IF NOT EXISTS
FOR (pt:EUPolicyTarget) REQUIRE pt.dateOfReport :: DATE
"""

statement_mpi="""
CREATE CONSTRAINT is_mpi IF NOT EXISTS
FOR ()-[mpi:IS_MPI_TO]-() REQUIRE mpi.date IS TYPED DATE
"""

In [11]:
statements = [statement_pt_comment,statement_pt_date,statement_pt_operator,statement_pt_unit,statement_pt_ref,
              statement_pt_unit,statement_pt_value,statement_pt_year,statement_mpi]

with driver.session(database=database_name) as session:
    for statement in statements:
        session.execute_write(create_constraint, statement)

#### Import the data 

In [6]:
## Policy Targets
statement_eu_policy_target = """
        MATCH (i:Indicator{code:$ind_code})
        MERGE (i)-[:HAS_POLICY_TARGET]->(pt:EUPolicyTarget{value:$val,unit:$un,operator:$oper,byYear:$year,
        reference:$ref,comment:$comm,dateOfReport:date($report_date)})
    """ 

## MPIs
statement_mpi = """ 
            MATCH (g:Goal{code:$goal_code})
            MATCH (i:Indicator{code:$ind_code})
            MERGE (i)-[:IS_MPI_TO{dateOfReport:date($report_date)}]->(g)
        """ 

## UN SDG Indicators
statement_eu_un = """ 
            MATCH (i1:Indicator{code:$eu_code})
            MATCH (i2:Indicator{code:$un_code})
            MERGE (i1)-[:ASSOCIATED_WITH{relevance:$association,dateOfReport:date($report_date)}]->(i2)
        """ 

with driver.session(database=database_name) as session:
    # EU_PolicyTargets
    for index, row in df_pt.iterrows():
        session.execute_write(write_eu_indicators, 
                                params_dict = {
                                    'ind_code':str(row['EU_ind']),
                                    'val':float(row['Value']),
                                    'un':str(row['Unit']),
                                    'oper':str(row['Operator']),
                                    'year':int(row['Year']),
                                    'ref':str(row['Reference']),
                                    'comm':str(row['Comments']),
                                    'report_date':str(row['DateOfReport'])},
                                statement = statement_eu_policy_target)  
    ## MPI
    for index, row in df_mpi.iterrows():
        session.execute_write(write_eu_indicators, 
                                params_dict = {
                                    'ind_code':str(row['eu_sdg']),
                                    'goal_code':str(row['goal_code']),
                                    'report_date':str(row['DateOfReport'])},
                                statement = statement_mpi)  
    ## UN-EU Association
    for index, row in df_eu_un.iterrows():
        session.execute_write(write_eu_indicators, 
                                params_dict = {
                                    'eu_code':str(row['eu_sdg']),
                                    'un_code':str(row['un_sdg']),
                                    'association':str(row['association']),
                                    'report_date':str(row['DateOfReport'])},
                                statement = statement_eu_un)  

In [ ]:
# specifically for sdg_04_40 which is connected to a policy target
# create new dimension code SCI_EF461_READ|A|T_A to average across the other three dimensions, for better aggregated visualisation
merge_query = """
    MATCH (i:Indicator {code: $indicator_code})--(s:Series)
    MERGE (newSm:SeriesMetadata {
        attributesCode: "PC",
        dimensionsCode: "SCI_EF461_READ|A|T_A",
        attributesDescription: "Percentage",
        dimensionsDescription: "Science_Math_Reading|Annual|Total_Average",
        seriesCode: $indicator_code
    })
    MERGE (s)-[:HAS_METADATA]->(newSm)
    WITH newSm, s, i
    MATCH (s)-[:HAS_METADATA]->(sm:SeriesMetadata)
    WHERE sm.dimensionsCode IN ["SCI|A|T", "EF461|A|T", "READ|A|T"]
    MATCH (sm)-[r:HAS_OBSERVATION]->(o:Observation)-[:REFERS_TO_AREA]->(ga:GeoArea)
    
    WITH DISTINCT newSm, i, o.time as time, ga, r.geoCode as geoCode, s
    
    // average for specific geo-time combination for each observation
    MATCH (s)-[:HAS_METADATA]->(sm2:SeriesMetadata)
    WHERE sm2.dimensionsCode IN ["SCI|A|T", "EF461|A|T", "READ|A|T"]
    MATCH (sm2)-[r2:HAS_OBSERVATION {time: time, geoCode: geoCode}]->(o2:Observation)
    
    WITH newSm, i, time, ga, geoCode, avg(o2.value) as avgValue
    MERGE (newObs:Observation {time: time)
    SET newObs.value = avgValue
    
    MERGE (newSm)-[:HAS_OBSERVATION {
        attributesCode: "PC",
        dimensionsCode: "SCI_EF461_READ|A|T_A",
        seriesCode: $indicator_code,
        time: time,
        geoCode: geoCode
    }]->(newObs)
    MERGE (newObs)-[:REFERS_TO_AREA]->(ga)
    MERGE (i)-[:HAS_OBSERVATIONS]->(ga)
    
    RETURN count(DISTINCT newObs) as mergedObservations
    """
    
with driver.session(database=database_name) as session:
        result = session.run(merge_query, indicator_code="sdg_04_40")
        record = result.single()
        merged_count = record["mergedObservations"]

> Check cypher query

In [13]:
records, summary, keys = driver.execute_query("""\
       MATCH (i:Indicator)-[r:COMES_FROM]->(s{name:'EU_SDG'}) WHERE i.code CONTAINS "sdg" RETURN count(i) as i
        """,routing_="r", database_=database_name)
print("{nodes_created} EU Indicators (expected {number} as of Report 2025) in {time} ms.".format(
    nodes_created=records[0]['i'],
    number=len(df),
    time=summary.result_available_after
))

records, summary, keys = driver.execute_query("""\
       MATCH (src:Source{name:'EU_SDG'})-[:COMES_FROM]-(eu_i:Indicator)-[a:ASSOCIATED_WITH]-(un_i:Indicator)-[:COMES_FROM]-(src1:Source{name:'UN_SDG'}) 
        return count(distinct eu_i.code) as i
        """,routing_="r",database_=database_name)
print("{nodes_created} EU Indicators associated with UN indicators (expected {number} as of Report 2025) in {time} ms.".format(  
    nodes_created=records[0]['i'],
    number=df_eu_un['eu_sdg'].nunique(),
    time=summary.result_available_after
))

records, summary, keys = driver.execute_query("""\
       MATCH(i:Indicator)-[m:IS_MPI_TO]-(g:Goal) return count(m) as i
        """,routing_="r",database_=database_name)
print("{nodes_created} EU MPIs (expected {number} as of Report 2025) in {time} ms.".format(
    nodes_created=records[0]['i'],
    number=len(df_mpi),
    time=summary.result_available_after
))

records = driver.execute_query("""\
       MATCH (g:Goal)--(i:Indicator)-[r:COMES_FROM]->(s{name:'EU_SDG'}) WHERE i.code CONTAINS "sdg" RETURN g.code as goal,count(i) as indicators
        """,routing_="r",database_=database_name,
    result_transformer_=neo4j.Result.to_df)
print("\nTotal Indicators per goal:")
print("{records}".format(
    records=records,
))

records = driver.execute_query("""\
       MATCH (g:Goal)<-[]-(i:Indicator)-[r:COMES_FROM]->(s{name:'EU_SDG'}) RETURN g.code as goal,count(i) as indicators
        """,routing_="r",database_=database_name,
    result_transformer_=neo4j.Result.to_df)
print("\nTotal MPIs per goal:")
print("{records}".format(
    records=records,
))

109 EU Indicators (expected 109 as of Report 2025) in 1 ms.
64 EU Indicators associated with UN indicators (expected 64 as of Report 2025) in 1 ms.
35 EU MPIs (expected 35 as of Report 2025) in 1 ms.

Total Indicators per goal:
   goal  indicators
0     1          11
1    10          11
2    11           9
3    12           8
4    13           8
5    14           6
6    15           9
7    16           6
8    17           6
9     2           9
10    3          11
11    4           7
12    5           8
13    6           8
14    7           7
15    8          11
16    9           9

Total MPIs per goal:
   goal  indicators
0     1           3
1    10           5
2    11           3
3    12           2
4    13           1
5    15           3
6     2           3
7     3           5
8     5           2
9     6           2
10    8           3
11    9           3


In [11]:
# check for sdg_04_40's extra dimension code specifically
records = driver.execute_query("""\
    MATCH (s:Series)-[:HAS_METADATA]->(sm:SeriesMetadata)
    WHERE sm.dimensionsCode IN ["SCI|A|T", "EF461|A|T", "READ|A|T"] AND sm.seriesCode = 'sdg_04_40' //counts the number of unique time-geoCode combinations
    MATCH (sm)-[r:HAS_OBSERVATION]->(o:Observation)
    WITH DISTINCT r.time as time, r.geoCode as geoCode
    WITH count(*) as expectedCount
    
    MATCH (newSm:SeriesMetadata {dimensionsCode: "SCI_EF461_READ|A|T_A", seriesCode: "sdg_04_40"})
    MATCH (newSm)-[r:HAS_OBSERVATION]->(o:Observation)
    
    RETURN expectedCount, count(o) as actualCount
    """, routing_="r", database_=database_name,
    result_transformer_=neo4j.Result.to_df
)
expected = records['expectedCount'].iloc[0]
actual = records['actualCount'].iloc[0]

print(f"Created {actual} nodes, expected {expected}")

Created 250 nodes, expected 250


### Import data from Eurostat Library

After that, by using the **preprocessing_eurostat_data** function we read the datasets regarding the EU_SDG indicators and preprocess them in order to get the desired output dataframe, which will be used as input by the function **import_sm_obs** that imports the data in the SustainGraph. 

The **preprocessing_eurostat_data** function:
- Drops NA values
- Create new columns about the dimensions and the attributes of each un sdg indicator

### Import National data

In [6]:
records, summary, keys = driver.execute_query("""\
        MATCH (r:Region{name:'Europe'})-[:HAS_SUBREGION]->(sr:SubRegion)-[:HAS_AREA]->(a:Area)
        MATCH (eu:EuropeanUnion)<-[:BELONGS_TO]-(eua:Area)
        WITH COLLECT(DISTINCT a.EUcode)+COLLECT(DISTINCT eua.EUcode) as geocodes
        UNWIND geocodes as codes
        RETURN COLLECT(DISTINCT codes) as geo
        """,routing_="r",database_=database_name)
available_areas = list(map(str,records[0]['geo']))

In [8]:
codes = ['sdg_08_10','sdg_13_40','sdg_07_40','sdg_06_60','sdg_14_60','sdg_15_20','sdg_14_10']

In [10]:
df_vi = df[df.Indicator_Code.isin(codes)].reset_index()

In [13]:
batch = 5000
isNUTS = False

# get the start time
st1 = time.time()

total_number_of_observations = 0
observations_per_indicator = {} 
series_metadata_expected = []

for index,row in df_vi.iterrows():   
    # preprocess eusgd indicators data
    df_eucode = preprocessing_eurostat_data(row.code,available_geoCodes = available_areas,is_local_level=isNUTS,filtering_params=None)

    if df_eucode is not None: 

        obs_count = len(df_eucode)
        total_number_of_observations += obs_count 
        observations_per_indicator[row.code] = obs_count 

        for _, r in df_eucode.iterrows():  
            series_metadata_expected.append({
                'indicator': row.code,
                'dim_code': r['dim_codes'],
                'att_code': r['att_codes']
            })
        
        st = time.time()

        print()
        # import data in the SustainGraph
        import_sm_obs(df_eucode,row.code,batch_size=batch,driver=driver,geoEUcode=True)

        # get the end time
        et = time.time()

        # get the execution time
        elapsed_time = et - st
        print('Time:', elapsed_time/60, 'minutes')
    
    if df_eucode is None:
        observations_per_indicator[row.code] = 0
        
# get the end time
et1 = time.time()

# get the execution time
elapsed_time_total = et1 - st1
print('Total execution time:', elapsed_time_total/60, 'minutes')


Indicator code:  sdg_06_60

Length of df before removing NA values: 888
Length of df after removing NA values: 887
Length of df after selecting only areas existing in neo4j SustainGraph: 816

Columns of df: ['unit', 'geo', 'freq', 'time', 'value']
Dimension columns: ['freq']

Attribute columns: unit

----------------------------------------------------------

816 observations: Done! (0.003565263748168945 minutes)
Time: 0.004731237888336182 minutes

Indicator code:  sdg_07_40

Length of df before removing NA values: 3276
Length of df after removing NA values: 3084
Length of df after selecting only areas existing in neo4j SustainGraph: 2892

Columns of df: ['unit', 'geo', 'nrg_bal', 'freq', 'time', 'value']
Dimension columns: ['freq', 'nrg_bal']

Attribute columns: unit

----------------------------------------------------------

2892 observations: Done! (0.008904147148132324 minutes)
Time: 0.012747430801391601 minutes

Indicator code:  sdg_08_10

Length of df before removing NA values:

In [16]:
records = driver.execute_query("""\
       MATCH (so:Source{name:'EU_SDG'})<-[:COMES_FROM]-(i:Indicator)--(s:Series)--(sm:SeriesMetadata)--(o:Observation)--(ga:Area)
       RETURN s.code as indicator, COUNT(DISTINCT o) as observations
       order by indicator
        """,routing_="r",database_=database_name,
    result_transformer_=neo4j.Result.to_df)
records

set_eu = set(df_eu['Indicator_Code'])
set_graph = set(records['indicator'])
exempt_from_graph = set_eu - set_graph 

print("EU SDG Indicators by Eurostat:", len(df_eu))
print("EU SDG Indicators with series information in the graph:", len(records))
print("EU SDG Indicators with no series information in the graph:", len(exempt_from_graph), ", listed:")
for i in exempt_from_graph:
    print(i)

EU SDG Indicators by Eurostat: 109
EU SDG Indicators with series information in the graph: 101
EU SDG Indicators with no series information in the graph: 8 , listed:
sdg_14_51
sdg_13_70
sdg_12_10
sdg_10_20
sdg_14_30
sdg_15_60
sdg_14_21
sdg_15_61


> Check cypher query

Checking for EU SDG Indicator observations that weren't imported properly to the graph ('mismatches' between graph and Eurostat database)

In [17]:
# checking for data that weren't imported correctly to the graph for each EU SDG Indicator per series metadata from Eurostat database, labeled 'mismatches'
check_statement = """ 
UNWIND $parameters AS row
MATCH (so:Source{name:'EU_SDG'})<-[:COMES_FROM]-(i:Indicator {code: row.code})
      --(s:Series)--(sm:SeriesMetadata)--(o:Observation)
WHERE i.code CONTAINS "sdg"
RETURN i.code AS indicator, 
       s.code AS series_code,
       sm.dimensionsCode AS dim_code, 
       sm.attributesCode AS att_code, 
       count(distinct o) AS imported
"""

check_results = []

for indicator in df['code'].unique():
    records, summary, keys = driver.execute_query(
        check_statement,
        parameters={"code": indicator},
        database_=database_name,
        routing_="r"
    )
    for r in records: 
        check_results.append({
            "indicator": r["indicator"],
            "series": r["series_code"],
            "dim_code": r["dim_code"],
            "att_code": r["att_code"],
            "series metadata": f"{r['dim_code']}|{r['att_code']}",
            "imported": r["imported"]
        })

expected_df = pd.DataFrame(series_metadata_expected)
expected_grouped = expected_df.groupby( #group expected counts per (indicator), (dim_code, att_code)->series metadata
    ['indicator', 'dim_code', 'att_code']
).size().reset_index(name='to_import')

actual_df = pd.DataFrame(check_results)
actual_df['series metadata'] = actual_df['dim_code'] + "|" + actual_df['att_code']

# merge dataframes
check_final_df = pd.merge(
    expected_grouped,
    actual_df,
    on=['indicator', 'dim_code', 'att_code'],
    how='outer'
).fillna(0)
check_final_df = check_final_df[
    ["indicator", "series", "dim_code", "att_code", "series metadata", "to_import", "imported"]
]

In [18]:
records = driver.execute_query("""\
    MATCH (so:Source{name:'EU_SDG'})<-[:COMES_FROM]-(i:Indicator)--(s:Series)--(sm:SeriesMetadata)--(o:Observation)--(ga:Area)
    WHERE i.code CONTAINS "sdg"
    RETURN count(distinct o) as observations
        """,routing_="r",database_=database_name,
    result_transformer_=neo4j.Result.to_df)
print('{} observations imported in total to SustainGraph,'.format(records['observations'][0]), "expected:", total_number_of_observations) 

143932 observations imported in total to SustainGraph, expected: 143932


In [19]:
mismatches = check_final_df[check_final_df['to_import'] != check_final_df['imported']]
if len(mismatches)==0:
    print("Mismatches: 0")
else:
    print("\nMismatches:\n", mismatches)

Mismatches: 0


In [20]:
check_final_df

,indicator,series,dim_code,att_code,series metadata,to_import,imported
0,sdg_01_10,sdg_01_10,TOTAL|A,PC,TOTAL|A|PC,320,320
1,sdg_01_10,sdg_01_10,TOTAL|A,THS_PER,TOTAL|A|THS_PER,320,320
2,sdg_01_10,sdg_01_10,Y_LT18|A,PC,Y_LT18|A|PC,320,320
3,sdg_01_10,sdg_01_10,Y_LT18|A,THS_PER,Y_LT18|A|THS_PER,320,320
4,sdg_01_10a,sdg_01_10a,DEG1|A,PC,DEG1|A|PC,322,322
...,...,...,...,...,...,...,...
265,sdg_17_40,sdg_17_40,A|GD|S13,MIO_EUR,A|GD|S13|MIO_EUR,675,675
266,sdg_17_40,sdg_17_40,A|GD|S13,PC_GDP,A|GD|S13|PC_GDP,675,675
267,sdg_17_50,sdg_17_50,A|ENV,PC_TSCO_X_ISCO,A|ENV|PC_TSCO_X_ISCO,712,712
268,sdg_17_60,sdg_17_60,A|VHCN_FX|DEG3,PC_HH,A|VHCN_FX|DEG3|PC_HH,186,186


### NUTS data

#### Retrieve NUTS data

In [21]:
records, summary, keys = driver.execute_query("""\
        MATCH (r:Region{name:'Europe'})-[:HAS_SUBREGION]->(sr:SubRegion)-[:HAS_AREA]->(a:Area)
        MATCH (eu:EuropeanUnion)<-[:BELONGS_TO]-(eua:Area)
        WITH COLLECT(DISTINCT a.EUcode)+COLLECT(DISTINCT eua.EUcode) as geocodes
        UNWIND geocodes as codes
        WITH COLLECT(DISTINCT codes) as geo
        MATCH (nuts3:NUTS3)<-[:HAS_NUTS3]-(nuts2:NUTS2)-[:HAS_NUTS2]-(nuts1:NUTS1)-[r:HAS_NUTS1]-(ar:Area)
        where ar.EUcode in geo
        RETURN COLLECT(DISTINCT nuts2.EUcode) + COLLECT(DISTINCT nuts3.EUcode) + COLLECT(DISTINCT nuts1.EUcode) as g
        """,routing_="r",database_=database_name)
available_nuts = list(map(str,records[0]['g']))

#### Associate NUTS regional dataset Indicator codes to EU SDG Indicator Codes as specified by Eurostat according to the relevant [Information Note](.\Data\3.EU_SDGs\Info-note_sdg_reg_20250206.pdf) .

In [22]:
# Read NUTS regional dataset Indicator codes, corresponding EU SDG Indicator codes from the Information Note pdf
# Note that the Information Note contains the same information 3 times, in English, German and French
reader = PdfReader("Data/3.EU_SDGs/Info-note_sdg_reg_20250206.pdf")

all_text = "" # initialise empty string to accummulate the text
for page in reader.pages:
    all_text += page.extract_text() # pass the text

# establish pattern to match EU SDG Indicator and NUTS Indicator codes as follows:
# r: raw string, to treat everything as a character
# EU SDG Indicator codes follow the patter of "sdg_xx_xx", x is an integer -> matched by sdg_\d{2}_\d{2})
# .*? matches the description in between, not needed for our dictionary
# NUTS Indicator codes are lowercase strings in the same line as EU SDG Indicator codes -> matched by [a-z]{2,}_[a-z0-9_]+: 2 or more lowercase letters, underscore, 1 or more lowercase letters, digits, or underscores
pattern = r"(sdg_\d{2}_\d{2}).*?\b([a-z]{2,}_[a-z0-9_]+)" 
matches = re.findall(pattern, all_text, re.DOTALL) # find matches, re.DOTALL to match newline characters too

codes_nuts_dict={dataset:indicator for indicator, dataset in matches} # create dictionary mapping

In [23]:
# Get description form toc_df for NUTS regional dataset Indicators
nuts_codes = []
corresponding_sdgs = []

for nuts_code, sdg_code in codes_nuts_dict.items():
    nuts_codes.append(nuts_code)
    corresponding_sdgs.append(sdg_code)

nuts_df = toc_df[toc_df['code'].isin(nuts_codes)][['code', 'title']] # NUTS Indicator codes from toc_df
nuts_to_sdg_mapping = dict(zip(nuts_codes, corresponding_sdgs))
nuts_df['sdg_code'] = nuts_df['code'].map(nuts_to_sdg_mapping)

first_dateofreport = df_eu['DateOfReport'].iloc[0] 
nuts_df['DateOfReport'] = first_dateofreport # add DateOfReport column from annual report
nuts_df

,code,title,sdg_code,DateOfReport
59,lfst_r_lfe2emprt,employment rates by nuts 2 region,sdg_08_30,2025-01-10
77,lfst_r_lfu2ltu,persons in long-term unemployment (12 months o...,sdg_08_40,2025-01-10
634,hlth_silc_08b_r,self-reported unmet needs for medical examinat...,sdg_03_60,2025-01-10
1161,nama_10r_2gdp,gross domestic product (gdp) at current market...,sdg_10_10,2025-01-10
1457,hlth_cd_asdr2,causes of death - standardised death rate by n...,sdg_16_10,2025-01-10
3389,ilc_li41,at-risk-of-poverty rate by nuts 2 region,sdg_01_20,2025-01-10
3475,ilc_lvhl21n,persons living in households with very low wor...,sdg_01_40,2025-01-10
3864,ilc_mdsd18,severe material and social deprivation by nuts...,sdg_01_31,2025-01-10
3957,ilc_peps11n,persons at risk of poverty or social exclusion...,sdg_01_10,2025-01-10
4196,edat_lfse_04,population in private households by educationa...,sdg_04_20,2025-01-10


In [24]:
def write_nuts_indicators(tx,statement, params_dict):
    tx.run(statement, parameters=params_dict)

In [25]:
# Import NUTS regional dataset Indicator codes, add 'corespondant' relationships with EU SDG indicator nodes, add COMES_FROM relationship with the EU_SDG node
parent_statement= """
    MATCH(pi:Indicator{code:$sdg_ind}), (i:Indicator{code:$nuts_ind}), (so:Source{name:'EU_SDG'})
    MERGE (i)-[:ASSOCIATED_WITH{relevance:'correspondant',dateOfReport:date($report_date)}]->(pi)
    MERGE (i)-[:COMES_FROM]->(so)
"""
statement_nuts_ind = """
    MERGE (i:Indicator{code:$nuts_ind,description:$desc})
    MERGE (s:Series{code:$nuts_ind,description:$desc,
                    dataProviderURL:"https://ec.europa.eu/eurostat/web/sdi/database"})
    MERGE (i)-[:HAS_SERIES]->(s)
    RETURN *
"""

i = 0
with driver.session(database=database_name) as session:
    for index,row in nuts_df.iterrows(): 
        session.execute_write(write_nuts_indicators, 
                            params_dict = {
                                'desc': str(row.title),
                                'nuts_ind': str(row.code)
                            },
                            statement = statement_nuts_ind)  
        i +=1
        session.execute_write(write_nuts_indicators, 
                        params_dict = { 
                            'nuts_ind': str(row.code),
                            'sdg_ind': str(row.sdg_code),
                            'report_date':str(row['DateOfReport'])
                            },
                        statement = parent_statement)

#### Import NUTS regional dataset data

In [26]:
# Import NUTS Indicator codes metadata
isNUTS = True
batch_size = 10000

number_of_nuts_observations = 0
observations_per_nuts_indicator = {}
series_metadata_expected_nuts = []

# if statement is commented out, needed in case there are NUTS indicators correlated to multiple EU SDG indicators
# filtering_params is commented out, needed in case specific filtering parameters are applied for each NUTS Indicator dataset

st1 = time.time()
for online_code, eu_code in codes_nuts_dict.items():
    # if isinstance(eu_code, list): 
    #     for i, element in enumerate(eu_code):
    #         #filtering_params_i = filter_params[online_code][i] if filter_params[online_code] else None #needed if filtering parameters are applied in NUTS datasets
    #         df_eucode = preprocessing_eurostat_data(online_code, available_geoCodes=available_nuts, is_local_level=isNUTS,
    #                                                       #filtering_params=filtering_params_i)
    #         )
    #         if df_eucode is not None:
    #             number_of_nuts_observations += len(df_eucode)
    #             st = time.time()
    #             import_sm_obs(df_eucode, online_code, batch_size=batch_size, driver=driver)
    #             et = time.time()
    #             elapsed_time = et - st
    #             print(f'Time for {online_code}-{element}: {elapsed_time/60:.2f} minutes')
    # else:
    #     #filtering_params_single = filter_params[online_code] if filter_params[online_code] else None #needed if filtering parameters are applied in NUTS datasets
        df_eucode = preprocessing_eurostat_data(online_code, available_geoCodes=available_nuts, is_local_level=isNUTS,
                                                      #filtering_params=filtering_params_single)
        )
        if df_eucode is not None:
            number_of_nuts_observations += len(df_eucode)
            observations_per_nuts_indicator[online_code] = len(df_eucode)

            for _, r in df_eucode.iterrows():  # check per series metadata, add dimensions dim_cod and att_code
                series_metadata_expected_nuts.append({
                'indicator': online_code,
                'dim_code': r['dim_codes'],
                'att_code': r['att_codes']
                })

            st = time.time()

            import_sm_obs(df_eucode, online_code, batch_size=batch_size, driver=driver)

            et = time.time()
            elapsed_time = et - st
            
            print(f'Time for {online_code}: {elapsed_time/60:.2f} minutes')

et1 = time.time()
elapsed_time_total = et1 - st1
print('Total execution time:', elapsed_time_total / 60, 'minutes')


Indicator code:  ilc_peps11n

Length of df before removing NA values: 3450
Length of df after removing NA values: 2342
Length of df after selecting only areas existing in neo4j SustainGraph: 2290

Columns of df: ['unit', 'geo', 'freq', 'time', 'value']
Dimension columns: ['freq']

Attribute columns: unit

----------------------------------------------------------
2290 observations: Done! (0.0582489291826884 minutes)
Time for ilc_peps11n: 0.06 minutes

Indicator code:  ilc_li41

Length of df before removing NA values: 7700
Length of df after removing NA values: 3746
Length of df after selecting only areas existing in neo4j SustainGraph: 3588

Columns of df: ['unit', 'geo', 'freq', 'time', 'value']
Dimension columns: ['freq']

Attribute columns: unit

----------------------------------------------------------
3588 observations: Done! (0.051470569769541424 minutes)
Time for ilc_li41: 0.06 minutes

Indicator code:  ilc_mdsd18

Length of df before removing NA values: 3795
Length of df afte

> Check cypher query

Checking for NUTS Indicator observations that weren't imported properly to the graph ('mismatches' between graph and Eurostat database)

In [27]:
records=driver.execute_query("""\
    MATCH (:Source {name: 'EU_SDG'})<-[:COMES_FROM]-(i1:Indicator)-[r:ASSOCIATED_WITH]-(i2:Indicator)
    WHERE i1.code CONTAINS "sdg" AND r.relevance='correspondant'
    RETURN COUNT(DISTINCT i2) AS total_i2_matched
    """,routing_="r",database_=database_name,result_transformer_=neo4j.Result.to_df)
print("NUTS regional dataset Indicator nodes created: {number}, expected: {expected} as of 2025.".format(
    number=records['total_i2_matched'].iloc[0],
    expected=len(nuts_df)
))

NUTS regional dataset Indicator nodes created: 19, expected: 19 as of 2025.


In [28]:
#Checking outgoing relationships for NUTS Indicators
records = driver.execute_query("""\
    MATCH (so:Source{name:'EU_SDG'})<-[:COMES_FROM]-(i:Indicator)-[:ASSOCIATED_WITH]-(i2:Indicator)--(s:Series)--(sm:SeriesMetadata)-[r:HAS_OBSERVATION]-(o:Observation)-[r1:REFERS_TO_AREA]->(ga)
    WHERE ((ga:NUTS1) or (ga:NUTS2) or (ga:NUTS3))
    RETURN s.code as indicator, COUNT(DISTINCT r1) as refer, COUNT(DISTINCT r) as has_obs,count(distinct o) as obs
    order by indicator
        """,routing_="r",database_=database_name,
    result_transformer_=neo4j.Result.to_df)

print("Individual indicator records:")
records

Individual indicator records:


,indicator,refer,has_obs,obs
0,aei_pr_soiler,375120,375120,375120
1,edat_lfse_04,507263,507263,507263
2,edat_lfse_16,23446,23446,23446
3,edat_lfse_22,101950,101950,101950
4,educ_uoe_enra22,2985,2985,2985
5,hlth_cd_asdr2,3461934,3461934,3461934
6,hlth_silc_08b_r,729,729,729
7,ilc_li41,3588,3588,3588
8,ilc_lvhl21n,2295,2295,2295
9,ilc_mdsd18,2164,2164,2164


In [29]:
# checking for data that weren't imported correctly to the graph for each NUTS Indicator per series metadata from Eurostat database, labeled 'mismatches'
check_df_nuts = pd.DataFrame(series_metadata_expected_nuts)
check_df_nuts['to_import'] = 1  
check_df_nuts['imported'] = 0   

# group expected counts per (indicator), (dim_code, att_code)->series metadata
expected_grouped_nuts = check_df_nuts.groupby(['indicator', 'dim_code', 'att_code']).size().reset_index(name='to_import')
expected_grouped_nuts['imported'] = 0  # initialize column to store actual counts

check_statement_nuts = """
UNWIND $parameters AS row
MATCH (i:Indicator {code: row.indicator})--(s:Series)--(sm:SeriesMetadata)--(o:Observation)
WHERE sm.dimensionsCode = row.dim_code AND sm.attributesCode = row.att_code
RETURN s.code AS series_code, count(distinct o) AS total
"""

results = []

# check imported counts per (indicator), (dim_code, att_code)->series metadata
for idx, row in expected_grouped_nuts.iterrows():
    params = {
        "indicator": str(row['indicator']),
        "dim_code": str(row['dim_code']),
        "att_code": str(row['att_code'])
    }
    records, summary, keys = driver.execute_query(
        check_statement_nuts,
        parameters=params,  
        database_=database_name,
        routing_="r"
    )
    for r in records:
        results.append({
            "indicator": row['indicator'],
            "dim_code": row['dim_code'],
            "att_code": row['att_code'],
            "to_import": row['to_import'],
            "series_code": r['series_code'],
            "imported": r['total']
        })

check_df_nuts_final = pd.DataFrame(results)

# add seriesmetadata column
check_df_nuts_final['seriesmetadata'] = check_df_nuts_final['dim_code'] + "|" + check_df_nuts_final['att_code']

# add eu sdg indicator column 
check_df_nuts_final['eu sdg indicator'] = check_df_nuts_final['indicator'].map(codes_nuts_dict)

check_df_nuts_final = check_df_nuts_final[
    ['indicator', 'eu sdg indicator', 'series_code', 'dim_code', 'att_code', 'seriesmetadata', 'to_import', 'imported']
]

In [30]:
total_observations_imported = check_df_nuts_final['imported'].sum()
total_observations_to_import = check_df_nuts_final['to_import'].sum()
print("Total UN SDG observations imported in SustainGraph", total_observations_imported, ", expected:",total_observations_to_import)

mismatches = check_df_nuts_final[check_df_nuts_final['to_import'] != check_df_nuts_final['imported']]
if len(mismatches)==0:
    print("Mismatches: 0")
else:
    print("\nMismatches:\n", mismatches)

Total UN SDG observations imported in SustainGraph 6898463 , expected: 6898463
Mismatches: 0


In [31]:
check_df_nuts_final

,indicator,eu sdg indicator,series_code,dim_code,att_code,seriesmetadata,to_import,imported
0,aei_pr_soiler,sdg_15_50,aei_pr_soiler,CLC23_321|A|MOD,HA,CLC23_321|A|MOD|HA,4689,4689
1,aei_pr_soiler,sdg_15_50,aei_pr_soiler,CLC23_321|A|MOD,KM2,CLC23_321|A|MOD|KM2,4689,4689
2,aei_pr_soiler,sdg_15_50,aei_pr_soiler,CLC23_321|A|MOD,PC,CLC23_321|A|MOD|PC,4689,4689
3,aei_pr_soiler,sdg_15_50,aei_pr_soiler,CLC23_321|A|MOD,T,CLC23_321|A|MOD|T,4689,4689
4,aei_pr_soiler,sdg_15_50,aei_pr_soiler,CLC23_321|A|MOD,T_HA,CLC23_321|A|MOD|T_HA,4689,4689
...,...,...,...,...,...,...,...,...
1475,tran_r_acci,sdg_11_40,tran_r_acci,A|KIL,NR,A|KIL|NR,11816,11816
1476,tran_r_acci,sdg_11_40,tran_r_acci,A|KIL,P_MHAB,A|KIL|P_MHAB,11526,11526
1477,trng_lfse_04,sdg_04_60,trng_lfse_04,Y25-64|A|F,PC,Y25-64|A|F|PC,8970,8970
1478,trng_lfse_04,sdg_04_60,trng_lfse_04,Y25-64|A|M,PC,Y25-64|A|M|PC,8887,8887
